# Building your first OpenAI Assistant

In [1]:
from dotenv import load_dotenv
import json
import logging
from openai import  AsyncOpenAI
from openai.types.beta import Thread
from openai.types.beta.threads import Run
from pydantic import BaseModel
from typing import Dict, List

In [23]:
load_dotenv(override=True)

logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

## 1. Define the Assistant class

In [24]:
class AssistantResult(BaseModel):
    response: str
    thread_id: str

class MaxTurnsReachedException(Exception):
    def __init__(self):
        super().__init__("Reached maximum number of turns")

In [25]:
class Assistant:
    def __init__(self, assistant_id: str, tools: Dict[str, callable]):
        self.client = AsyncOpenAI()
        self.assistant_id = assistant_id
        self.assistant = None
        self.tools = tools
    
    async def retrieve_assistant(self):
        if self.assistant is None:
            self.assistant = await self.client.beta.assistants.retrieve(self.assistant_id)
        return self.assistant
    
    async def run(self, query: str, thread_id: str = None, max_turns: int = 5) -> AssistantResult:
        await self.retrieve_assistant()

        log.info("Running assistant with thread_id %s", thread_id)

        if(thread_id is None):
            thread: Thread = await self.client.beta.threads.create()
            log.info("Created new thread with id %s", thread.id)
        else:
            thread: Thread = await self.client.beta.threads.retrieve(thread_id)
            log.info("Retrieved thread with id %s", thread.id)
        
        log.info("Sending query to thread %s", thread.id)
        await self.client.beta.threads.messages.create(
            thread_id=thread.id, role="user", content=query
        )
        log.info("Query sent to thread %s", thread.id)

        log.info("Creating and polling run for thread %s", thread.id)
        run: Run = await self.client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
        )
        log.info("Run created and polled for thread %s", thread.id)

        for turn in range(max_turns):

            # Fetch the last message from the thread
            log.info("Fetching last message from thread %s", thread.id)
            messages = await self.client.beta.threads.messages.list(
                thread_id=thread.id,
                run_id=run.id,
                order="desc",
                limit=1,
            )
            log.info("Fetched last message from thread %s", thread.id)

            # Check for the terminal state of the Run.
            # If state is "completed", exit agent loop and return the LLM response.
            if run.status == "completed":
                log.info("Run completed for thread %s", thread.id)
                assistant_res: str = next(
                    (
                        content.text.value
                        for content in messages.data[0].content
                        if content.type == "text"
                    ),
                    None,
                )

                return AssistantResult(thread_id=thread.id, response=assistant_res)
            
            # If state is "requires_action", function calls are required. Execute the functions and send their outputs to the LLM.
            if run.status == "requires_action":
                log.info("Run requires action for thread %s", thread.id)
                func_tool_outputs = []

                # LLM can ask for multiple functions to be executed. Execute all function calls in loop and
                # append the results into `func_tool_outputs` list.
                for tool in run.required_action.submit_tool_outputs.tool_calls:
                    # parse the arguments required for the function call from the LLM response
                    args = (
                        json.loads(tool.function.arguments)
                        if tool.function.arguments
                        else {}
                    )

                    try:
                        log.info("Running function %s with args %s for thread %s", tool.function.name, args, thread.id)
                        func_output = await self.tools[tool.function.name](**args)
                        log.info("Function %s executed successfully for thread %s", tool.function.name, thread.id)
                    except Exception as e:
                        log.error("Error in running function %s: %s", tool.function.name, e)
                        func_output = f'Error in running function {tool.function.name}: {e}'

                    # OpenAI needs the output of the function call against the tool_call_id
                    func_tool_outputs.append(
                        {"tool_call_id": tool.id, "output": str(func_output)}
                    )

                # Submit the function call outputs back to OpenAI
                log.info("Submitting function outputs for thread %s", thread.id)
                run = await self.client.beta.threads.runs.submit_tool_outputs_and_poll(
                    thread_id=thread.id, run_id=run.id, tool_outputs=func_tool_outputs
                )
                log.info("Function outputs submitted for thread %s", thread.id)

                # Continue the agent loop.
                # Agent will check the output of the function output submission as part of next iteration.
                continue

            # Handle errors if terminal state is "failed"
            else:
                if run.status == "failed":
                    log.error(
                        f"OpenAIFunctionAgent turn-{turn+1} | Run failure reason: {run.last_error}"
                    )

                raise Exception(
                    f"Failed to generate text due to: {run.last_error}"
                )
        
        # Raise error if turn-limit is reached.
        await self.client.beta.threads.runs.cancel(run.id,thread_id=thread_id)
        raise MaxTurnsReachedException()
    
    async def cancel_thread_run(self, thread_id: str):
        thread: Thread = await self.client.beta.threads.retrieve(thread_id)
        run: Run = await self.client.beta.threads.runs.list(thread_id=thread.id).data[0]

        await self.client.beta.threads.runs.cancel(run.id,thread_id=thread_id)

## 2. Define the Assistant Tools

In [26]:
async def get_current_temperature(location: str, unit: str):
    return 30

## 3. Create the OpenAI Assistant using the OpenAI Playground

1. Navigate to [OpenAI Assistant Playground](https://platform.openai.com/playground/assistants)
2. Create a new Assistant and name following the format: `<name_first_letter>_<surname>_first_assistant`
3. Define the System Instructions
4. Select gpt-4o-mini as the model
5. Generate a new Function definition

## 4. Instantiate the Assistant

In [27]:
ASSISTANT_ID="<YOUR_ASSISTANT_ID>"
TOOLS = {"get_current_temperature": get_current_temperature}

In [28]:
assistant = Assistant(ASSISTANT_ID, TOOLS)

## 5. Use the Assistant

In [ ]:
input = "What is the temperature in New York?"

response = await assistant.run(query=input, max_turns=5)

In [ ]:
print(response)

In [ ]:
input = "and in Paris?"

response = await assistant.run(query=input, thread_id=response.thread_id, max_turns=5)

In [ ]:
print(response)